In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import torch.nn as nn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
dataset = load_dataset('imdb')

In [ ]:
len(dataset['train']['label'])


In [ ]:
texts_array_train = dataset['train']['text']
labels_array_train = dataset['train']['label']
texts_array_test = dataset['test']['text']

In [ ]:
vectorizer = CountVectorizer(max_features=10000,
                             min_df=7, max_df=0.85,
                             binary=True,
                             stop_words='english')


In [ ]:
vectorizer.fit(texts_array_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    texts_array_train, labels_array_train, test_size=0.15,
    random_state=42, stratify=labels_array_train

)

In [ ]:
class BoWDataset(Dataset):
    def __init__(self, embed, label):
        self.embed = torch.FloatTensor(embed)
        self.label = torch.FloatTensor(label)


    def __getitem__(self, index):
        embed = self.embed[index]

        return {
            'embed': embed ,
            'y': self.label[index]
        }




    def __len__(self):
        return len(self.label)
        



In [ ]:
X_train = vectorizer.transform(X_train).toarray()

In [ ]:
X_val = vectorizer.transform(X_val).toarray()

In [ ]:
train_dataset = BoWDataset(X_train, y_train)

In [ ]:
val_dataset = BoWDataset(X_val, y_val)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10000, 1)

    def forward(self, x):
        return self.linear(x).squeeze(1)
    

In [ ]:
model = RegressionModel()

criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model.to(device)

In [ ]:
epochs = 15

In [ ]:
i = next(iter(train_dataloader))

In [ ]:
i['y']

In [ ]:
for epoch in range(epochs):
    
    model.train()
    
    train_loss = 0
    

    for batch in train_dataloader:
        X = batch['embed'].to(device)
        y = batch['y'].to(device)

        optimizer.zero_grad()

        logits = model(X)

        loss = criterion(logits, y)

        loss.backward()

        optimizer.step()

        train_loss += loss.item()

    train_loss /=len(train_dataloader)
    

    model.eval()
    
    val_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            X = batch['embed'].to(device)
            y = batch['y'].to(device)

            logits = model(X)
            loss = criterion(logits, y)
            
            val_loss += loss.item()
    val_loss /=len(val_dataloader)




    print(f'Epoch:{epoch}')
    print(f'Train loss: {train_loss:.4f}')
    print(f'Val loss: {val_loss:.4f}')
    

In [ ]:
from sklearn.metrics import accuracy_score



In [ ]:
X_test = dataset['test']['text']
X_test = vectorizer.transform(X_test).toarray()
y_test = dataset['test']['label']

In [ ]:
test_dataset = BoWDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=64)

In [ ]:
all_preds = []
all_labels = []

In [ ]:
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        X = batch['embed'].to(device)
        y = batch['y'].to(device)

        outputs = model(X)


        preds = (torch.sigmoid(outputs)>0.5).long()

        all_preds.append(preds.cpu())
        all_labels.append(y.cpu())

In [ ]:
all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

In [ ]:
print(accuracy_score(all_preds, all_labels))